In [2]:
!apt update && apt install -y curl git

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,321 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,664 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,649 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu

In [3]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [17]:
!ollama pull llama3


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏  10 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  37 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   2% ▕▏  85 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   3% ▕▏ 124 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   3% ▕▏ 148 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   4% ▕▏ 174 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 210 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 229 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 253 MB/

In [29]:
!pkill -f ngrok  # Kills all ngrok processes
!fuser -k 8501/tcp  # Kills any process using port 6000 (or use 5000 if needed)
# Install required libraries
OLLAMA_API_URL = "http://127.0.0.1:11434"
!nohup ollama serve > /dev/null 2>&1 &



In [19]:
!pip install streamlit PyPDF2 requests python-docx



In [20]:
!pip install pyngrok

In [30]:
from pyngrok import ngrok

# Replace with your ngrok auth token
NGROK_AUTH_TOKEN = "2WG8G2YZNV9jofWBKu5B2E6hA3Q_719vVw9PLU1SwQLwCtcNd"

# Set authentication token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ ngrok Auth Token Set!")

public_url = ngrok.connect(8501).public_url
print(f"🚀 Streamlit App is running at: {public_url}")


✅ ngrok Auth Token Set!
🚀 Streamlit App is running at: https://a421-34-19-21-241.ngrok-free.app


In [27]:
import requests

OLLAMA_API_URL = "http://127.0.0.1:11434/api/generate"

headers = {"Content-Type": "application/json"}
payload = {
    "model": "llama3",
    "prompt": "What is the capital of France?",
    "stream": False
}

response = requests.post(OLLAMA_API_URL, json=payload, headers=headers)

if response.status_code == 200:
    print("✅ Response from Ollama:", response.json()["response"])
else:
    print(f"❌ Error from Ollama: {response.status_code}, {response.text}")


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fccce193510>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [31]:
%%writefile app.py
import streamlit as st
import requests
from bs4 import BeautifulSoup
from googlesearch import search

# Streamlit UI Setup
st.set_page_config(page_title="Recipe Finder", layout="wide")
st.title("🍽️ Recipe Finder - Get Recipes Using Your Ingredients")
st.caption("Enter ingredients you have, and get the best recipes!")

# User Input for Ingredients
ingredients = st.text_input("📝 Enter Ingredients (comma-separated)", placeholder="e.g., chicken, garlic, onion, tomato")

def get_recipe_links(ingredients):
    """Searches Google for the top 5 recipe websites based on ingredients"""
    query = f"recipes with {ingredients}"
    try:
        return list(search(query, num=5, stop=5, pause=2))
    except Exception as e:
        return ["Error fetching recipes: " + str(e)]

if ingredients:
    st.info("🔍 Searching for recipes...")
    recipe_links = get_recipe_links(ingredients)

    if "Error" in recipe_links[0]:
        st.error(recipe_links[0])
    else:
        st.success("✅ Found top 5 recipes!")
        for i, link in enumerate(recipe_links, start=1):
            st.markdown(f"### 🍽️ Recipe {i}: [View Recipe]({link})")

st.markdown("---")
st.caption("Powered by Google Search & Web Scraping 🍽️")


Overwriting app.py


In [ ]:
!streamlit run app.py &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.19.21.241:8501

